In [ ]:
import json
import requests
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point, Polygon
import shapefile as shp
import gmplot
import calendar
import datetime
%matplotlib inline

from pandas.io.json import json_normalize
from requests import request

In [ ]:
#Backlog: create logic to get links and build list of URLs from Chicago Open Data
#Chicago Open data developers: http://dev.cityofchicago.org/open%20data/2019/07/17/scooter-gbfs-public-feeds.html

bird = "https://mds.bird.co/gbfs/chicago/free_bike_status.json"
bolt = "https://www.bolt.miami/bolt2/chi/gbfs/en/free_bike_status.json"
gruv = "https://portal.clevrmobility.com/api/gbfs/chicago/en/free_bike_status/?format=json"
jump = "https://gbfs.uber.com/v1/chicago/free_bike_status.json"
lime = "https://data.lime.bike/api/partners/v1/gbfs/chicago/free_bike_status"
lyft = "https://s3.amazonaws.com/lyft-lastmile-production-iad/lbs/chi/free_bike_status.json"
sherpa = "https://mds.bird.co/gbfs/platform-partner/sherpa/chicago/free_bike_status.json"
spin = "https://web.spin.pm/api/gbfs/v1/chicago/free_bike_status"
wheels = "https://chicago-gbfs.getwheelsapp.com/free_bike_status.json"
veoride = "https://share.veoride.com/api/share/gbfs/free_bike_status"

#Backlog: gruv and veoride excluded for now - no data provided, add conditions check during get/response
scooter_url = [bird, bolt, jump, lime, lyft, sherpa, spin, wheels] #, gruv, veoride]
scooter_url

In [ ]:
#Backlog: Create arrays and iterate over a loop to get all source URLs and read into single data frame
#Backlog: Create logic to refresh data periodically 

#ref: https://stackoverflow.com/questions/16129652/accessing-json-elements

response = requests.get(scooter_url[0]).json()
data = response['data']
df1 = json_normalize(data, 'bikes', errors='ignore')

response = requests.get(scooter_url[1]).json()
data = response['data']
df2 = json_normalize(data, 'bikes', errors='ignore')

response = requests.get(scooter_url[2]).json()
data = response['data']
df3 = json_normalize(data, 'bikes', errors='ignore')

response = requests.get(scooter_url[3]).json()
data = response['data']
df4 = json_normalize(data, 'bikes', errors='ignore')

response = requests.get(scooter_url[4]).json()
data = response['data']
df5 = json_normalize(data, 'bikes', errors='ignore')

response = requests.get(scooter_url[5]).json()
data = response['data']
df6 = json_normalize(data, 'bikes', errors='ignore')

response = requests.get(scooter_url[6]).json()
data = response['data']
df7 = json_normalize(data, 'bikes', errors='ignore')

response = requests.get(scooter_url[7]).json()
data = response['data']
df8 = json_normalize(data, 'bikes', errors='ignore')


In [ ]:
#Backlog: group DFs by source or manufacturer type, i.e. scooter co. 1, scooter co. 2, scooter co. 3
#Backlog: add headers for each DF group
#Backlog: Merge with prior cell and execute concat in the loop

df_superset = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8], sort="False")
df_superset

#df_superset['lon'].dtype #testing lat and lon datatypes

#Ref: https://cmdlinetips.com/2018/09/how-to-change-data-type-for-one-or-more-columns-in-pandas-dataframe/

#Backlog: lat/lon fields are str for some reason; correct by changing concat'd df to floats
df_superset.lat = df_superset.lat.astype(float)
df_superset.lon = df_superset.lon.astype(float)

In [ ]:
#turn lat / lon float values into geometric data
geometry = [Point(xy) for xy in zip(df_superset.lon, df_superset.lat)]

#verify a few entires and length 
geometry[:3]
len(geometry)

In [ ]:
crs = {'init': 'epsg:4326'}
geo_df = gpd.GeoDataFrame(df_superset, crs = crs, geometry = geometry)
geo_df.head()

In [ ]:
#ref: https://towardsdatascience.com/geopandas-101-plot-any-data-with-a-latitude-and-longitude-on-a-map-98e01944b972
#ref: https://data.cityofchicago.org/Transportation/Street-Center-Lines/6imu-meau

street_map = gpd.read_file('/Users/cheese/lData/Scooters/geo_export_065bfbf1-f9d4-49af-8c7f-ce5e73f3bc8f.shp')
fig,ax = plt.subplots(figsize = (15, 15))
street_map.plot(ax = ax, alpha =0.4, color="grey")

#Add information to show the as-of date of the data
plt.title("Chicago Scooter Snapshot at " + str(datetime.datetime.now()))

geo_df.plot(ax = ax, markersize = 20, color = "red", marker = "o", label = "scooter")
plt.savefig('map.png')

#Backlog: find a better basemap shapefile based on the data plot for west chicago
#Backlog: after identifying plots by source company, change markers, labels, and legend; or keep it as one bunch?
#Backlog: integrate with gmplot to create heatmaps as an alternative visual
#Backlog: Explore time series analysis 
#Backlog: Embed to website and provide real-time / near real-time data visualization

#data use and terms: https://www.chicago.gov/city/en/depts/cdot/supp_info/escooter-share-pilot-project.html
#data use and terms: https://www.chicago.gov/content/dam/city/depts/cdot/Misc/EScooters/EScootersPilotProgramTerms_06-07-19.pdf
#data use and terms: https://github.com/CityOfLosAngeles/mobility-data-specification
#this code and analysis for educational purposes only